# Imports and Preprocessing

In [57]:
import os
import sys
import pandas as pd

folder_path = os.path.join(os.path.dirname(os.getcwd()), 'Data_Test_Multi_Raw')
print(folder_path)
file_names = ['data_test.csv', 'data_train.csv', 'target_test.csv', 'target_train.csv']

data_frames = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    data_frames.append(df)

data_test = data_frames[0]
data_train = data_frames[1]
target_test = data_frames[2]    
target_train = data_frames[3]

print(data_train.shape)
print(target_train.shape)

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\Data_Test_Multi_Raw
(800, 6)
(800, 6)


# Preprocessing

In [58]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Define preprocessing for numeric columns (scale them)
numeric_features = ['temperature']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Define preprocessing for categorical features (encode them)
categorical_features = ['location']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)],
    remainder='passthrough')

# Preprocessing on train data
X_train = preprocessor.fit_transform(data_train)

# Preprocessing on test data
X_test = preprocessor.transform(data_test)


print(X_train.shape)
print(X_test.shape)

for column in target_train.columns:
    print(target_train[column])

(800, 15)
(200, 15)
0      59
1      49
2      59
3      59
4      45
       ..
795    45
796    45
797    55
798    49
799    45
Name: product_1_demand, Length: 800, dtype: int64
0      73
1      60
2      73
3      73
4      55
       ..
795    55
796    55
797    69
798    61
799    55
Name: product_2_demand, Length: 800, dtype: int64
0      63
1      52
2      63
3      63
4      48
       ..
795    48
796    48
797    59
798    52
799    48
Name: product_3_demand, Length: 800, dtype: int64
0      35
1      29
2      35
3      35
4      28
       ..
795    28
796    28
797    31
798    28
799    28
Name: product_4_demand, Length: 800, dtype: int64
0      81
1      67
2      81
3      81
4      61
       ..
795    61
796    61
797    77
798    68
799    61
Name: product_5_demand, Length: 800, dtype: int64
0      84
1      69
2      84
3      84
4      63
       ..
795    63
796    63
797    79
798    70
799    63
Name: product_6_demand, Length: 800, dtype: int64


# Demand Forecasting with ANN Model

In [59]:
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.optimizers import Adam
from scipy.stats import reciprocal
import numpy as np

# Model creation function 
def create_model(n_hidden, n_neurons, learning_rate, activation):
    model = Sequential()
    # Input Layer
    model.add(Dense(n_neurons,input_dim=15, activation=activation))
    # Hidden Layer
    for _ in range(n_hidden):
        model.add(Dense(n_neurons, activation=activation))
    # Output Layer
    model.add(Dense(1))
    model.compile(loss="mean_squared_error", optimizer=Adam(learning_rate=learning_rate))
    return model

# Model builder function 
def model_builder(n_hidden, n_neurons, learning_rate, activation, batch_size, epochs):
    return KerasRegressor(build_fn=create_model, verbose=0, n_hidden=n_hidden, n_neurons=n_neurons, 
                          learning_rate=learning_rate, activation=activation, batch_size=batch_size, epochs=epochs)

# Define the hyperparameters to search for the Randomized Search
params = {
    "n_hidden": range(0, 15),
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(1e-4, 1e-2),
    "batch_size": [16, 32, 64, 128],
    "epochs": [10,15, 20, 25, 30],
    "activation": ['relu', 'sigmoid', 'tanh'] #
}



# Iterate over all items and predict values
target_pred_ANN = pd.DataFrame()
train_pred_ANN = pd.DataFrame()

for column in target_train.columns:
    # Create the model and define hyperparameter tuning with Randomized Search
    model_ANN = model_builder(1,30,3e-3,'relu', 32, 20)
    rnd_search_cv_ANN = RandomizedSearchCV(model_ANN, param_distributions=params, n_iter=10, cv=3, scoring='neg_mean_squared_error')

    # Fit the model on the training data
    rnd_search_cv_ANN.fit(X_train, target_train[column])
    print(rnd_search_cv_ANN.best_params_)

    # Predict the target values and add them as new columns to the DataFrames
    target_pred_ANN[column] = rnd_search_cv_ANN.predict(X_test)
    train_pred_ANN[column] = rnd_search_cv_ANN.predict(X_train)

# Round the results
target_pred_ANN = target_pred_ANN.round()
train_pred_ANN = train_pred_ANN.round()

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

{'activation': 'relu', 'batch_size': 32, 'epochs': 15, 'learning_rate': 0.000630794573045983, 'n_hidden': 0, 'n_neurons': 70}


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

{'activation': 'relu', 'batch_size': 64, 'epochs': 30, 'learning_rate': 0.00026789324798979467, 'n_hidden': 13, 'n_neurons': 23}


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

...

{'activation': 'relu', 'batch_size': 32, 'epochs': 25, 'learning_rate': 0.00011003077063814418, 'n_hidden': 10, 'n_neurons': 38}


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

{'activation': 'relu', 'batch_size': 64, 'epochs': 30, 'learning_rate': 0.0010176999510945874, 'n_hidden': 9, 'n_neurons': 37}


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

{'activation': 'relu', 'batch_size': 128, 'epochs': 25, 'learning_rate': 0.0036959593175434854, 'n_hidden': 13, 'n_neurons': 42}


# Demand Forecasting with DT (LightGBM)

In [60]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# Create the model and define the hyperparameters to search for the Randomized Search
model_DT = lgb.LGBMRegressor()
param_distribs = {
    "n_estimators": [50, 100, 200],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.1, 0.5],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
}

# Create the Randomized Search object
rnd_search_cv_DT = RandomizedSearchCV(model_DT, param_distribs, n_iter=10, cv=3, scoring='neg_mean_squared_error')

# Iterate over all items and predict values
target_pred_DT = pd.DataFrame()
train_pred_DT = pd.DataFrame()

for column in target_train:

    # Fit the model on the training data
    rnd_search_cv_DT.fit(X_train, target_train[column])
    print(rnd_search_cv_ANN.best_params_)

    # Get the best parameters and best estimator
    best_params = rnd_search_cv_DT.best_params_
    best_estimator = rnd_search_cv_DT.best_estimator_

    # Predict the target values and add them as new columns to the DataFrames
    target_pred_DT[column] = best_estimator.predict(X_test)
    train_pred_DT[column] = best_estimator.predict(X_train)

# Round the predicted values
train_pred_DT = train_pred_DT.round(0)
target_pred_DT = target_pred_DT.round(0)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000400 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 80
[LightGBM] [Info] Number of data points in the train set: 533, number of used features: 14
[LightGBM] [Info] Start training from score 49.352720
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further split

# Solving Newsvendor (Non-parametric)

In [104]:
import numpy as np

# Initialize an empty list to store the final order quantities
final_order_quantities_ANN = pd.DataFrame(target_pred_ANN)
final_order_quantities_DT = pd.DataFrame(target_pred_DT)

# Parameters for multi-item newsvendor problem
prices = np.array([0.3, 0.5, 0.6, 0.5, 0.5, 0.5]) #price data
costs = np.array([0.06, 0.06, 0.06, 0.06, 0.06, 0.06]) #cost data
salvages = np.array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01]) #salvage data
underage_data = prices - costs 
overage_data = costs - salvages 


alpha_data = np.array([             #alpha data
    [0.0, 0.1, 0.05, 0.1, 0.05, 0.1],
    [0.15, 0.0, 0.1, 0.05, 0.05, 0.05],
    [0.1, 0.2, 0.0, 0.05, 0.1, 0.05],
    [0.05, 0.05, 0.05, 0.0, 0.15, 0.2],
    [0.1, 0.05, 0.15, 0.2, 0.0, 0.05],
    [0.05, 0.1, 0.05, 0.15, 0.1, 0.0]
])


In [118]:
import pulp

def multi_product_newsvendor(n, d, prices, costs, salvages, alpha_data):
    # Define the problem
    model = pulp.LpProblem("Multi_Product_Newsvendor", pulp.LpMaximize)

    # Index for products
    products = range(n)  # Assume n products

    # Decision Variables
    q = pulp.LpVariable.dicts("q", products, lowBound=0, cat='Continuous')

    # Parameters 
    p = list(map(float, prices))  # Prices
    c = list(map(float, costs))  # Costs
    s = list(map(float, salvages))  # Salvage values
    alpha = {(i, j): alpha_data[i, j] for i in range(alpha_data.shape[0]) for j in range(alpha_data.shape[1])}

    # Objective Function
    model += pulp.lpSum([(p[i] - c[i]) * q[i] - (p[i] - s[i]) * (q[i] - d[i] - pulp.lpSum([alpha[(j, i)] * (d[j] - q[j]) for j in products if j != i])) for i in products])

    # Constraints
    for i in products:
        model += q[i] >= d[i] + pulp.lpSum([alpha[(j, i)] * (d[j] - q[j]) for j in products if j != i])

    # Solve the model
    model.solve()

    # Convert the decision variables to a DataFrame
    order_quantities = {i: pulp.value(q[i]) for i in range(n)}

    return order_quantities

In [134]:
import pulp as pl

def solve_milp_single_period(num_products, prices, costs, salvages, demands, substitution_rates):
    # Constants
    u = [prices[i] - costs[i] for i in range(num_products)]
    o = [costs[i] - salvages[i] for i in range(num_products)]
    M = 100  # Large number for big-M method

    # Create the problem variable to maximize the objective
    problem = pl.LpProblem("Single_Period_Newsvendor", pl.LpMaximize)

    # Decision variables
    q = pl.LpVariable.dicts("OrderQty", (range(num_products)), lowBound=0, cat='Continuous')
    y = pl.LpVariable.dicts("Overage", (range(num_products)), lowBound=0, cat='Continuous')
    v = pl.LpVariable.dicts("Underage", (range(num_products)), lowBound=0, cat='Continuous')
    z = pl.LpVariable.dicts("BinaryVar", (range(num_products)), cat='Binary')

    # Objective Function
    problem += pl.lpSum(u[i] * q[i] - (u[i] + o[i]) * y[i] for i in range(num_products))

    # Constraints
    for i in range(num_products):
        # Substitution demand constraint
        problem += y[i] >= q[i] - demands[i] - pl.lpSum(substitution_rates[j][i] * v[j] for j in range(num_products) if j != i)
        
        # Demand satisfaction constraints
        problem += v[i] <= demands[i] - q[i] + M * z[i]
        problem += v[i] >= demands[i] - q[i] - M * (1 - z[i])


    # Solve the problem
    problem.solve()

    # Output the results
    order_quantities = {i: pl.value(q[i]) for i in range(num_products)}
    return order_quantities



In [135]:
forecast_error_ANN = target_train - train_pred_ANN
demand_scenarios = pd.DataFrame(target_pred_ANN.iloc[1] + forecast_error_ANN.iloc[1])
demand_scenarios = demand_scenarios.T
print(demand_scenarios)

print(type(alpha_data))
y = solve_milp_single_period(6, prices, costs, salvages, demand_scenarios.iloc[0].tolist(), alpha_data.tolist())
print(y)
x = multi_product_newsvendor(6, demand_scenarios.iloc[0].tolist(), prices.tolist(), costs.tolist(), salvages.tolist(), alpha_data)
print(x)

   product_1_demand  product_2_demand  product_3_demand  product_4_demand  \
1              47.0              60.0              52.0              29.0   

   product_5_demand  product_6_demand  
1              66.0              69.0  
<class 'numpy.ndarray'>
{0: 77.598923, 1: 95.141238, 2: 79.012391, 3: 67.324995, 4: 96.29775, 5: 98.65254}
{0: 47.0, 1: 60.0, 2: 52.0, 3: 29.0, 4: 66.0, 5: 69.0}


In [63]:
import pandas as pd
import numpy as np
from scipy.stats import norm

scenario_size = 100
demand_scenarios = pd.DataFrame()

# Initialize an empty list to store the final order quantities
final_order_quantities_ANN = []

# Calculate the forecast error for ANN on the prediction of the training data
forecast_error_ANN = target_train - train_pred_ANN
print(forecast_error_ANN.shape)

# Loop over each week in data_test
for i in range(len(X_test)):

    # Create Demand Scenarios for this week
    random_row_index = np.random.randint(0, len(forecast_error_ANN))
    demand_scenarios = pd.DataFrame([target_pred_ANN.iloc[j] + forecast_error_ANN.iloc[random_row_index] for j in range(scenario_size)])
    print(demand_scenarios)
        
# Initialize a list to store the solutions for each scenario
saa_solutions = pd.DataFrame()

# For each demand scenario, solve the newsvendor problem
for i in range(len(demand_scenarios)):
    
    # Calculate the solution for this scenario
    solution = multi_product_newsvendor(6, demand_scenarios.iloc[i], prices, costs, salvages, alpha_data)

    # Store the solution for this scenario
    saa_solutions = saa_solutions.append(solution, ignore_index=True)

# Average the solutions to get the final allocation
final_allocation = saa_solutions.mean()

# Store the final order quantities
final_order_quantities_ANN.append(pd.DataFrame(final_allocation))

(800, 6)
    product_1_demand  product_2_demand  product_3_demand  product_4_demand  \
0               50.0              60.0              51.0              27.0   
1               49.0              60.0              51.0              27.0   
2               50.0              60.0              51.0              27.0   
3               52.0              60.0              51.0              27.0   
4               48.0              60.0              51.0              27.0   
..               ...               ...               ...               ...   
95              51.0              60.0              51.0              27.0   
96              48.0              60.0              51.0              27.0   
97              51.0              60.0              51.0              27.0   
98              48.0              60.0              51.0              27.0   
99              51.0              60.0              51.0              27.0   

    product_5_demand  product_6_demand  
0            

C:\Users\lanza\AppData\Local\Temp\ipykernel_24560\1848351629.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  model += pulp.lpSum([(p[i] - c[i]) * q[i] - (p[i] - s[i]) * (q[i] - d[i] - pulp.lpSum([alpha[(j, i)] * (d[j] - q[j]) for j in products if j != i])) for i in products])


AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm

scenario_size = 100

# Initialize an empty list to store the final order quantities
final_order_quantities_DT = []

# Calculate the forecast error for ANN on the prediction of the training data
target_train_flatten = target_train.values.flatten()
train_pred_DT_flatten = train_pred_DT.flatten()
forecast_error_DT = target_train_flatten - train_pred_DT_flatten
forecast_error_DT_std = np.std(forecast_error_DT)

# Loop over each week in data_test
for i in range(len(data_test)):

    # Create Demand Scenarios for this week
    demand_scenarios = target_pred_ANN[i] + np.random.choice(forecast_error_ANN, size=scenario_size) 

    # Initialize a list to store the solutions for each scenario
    saa_solutions = []

    # For each demand scenario, solve the newsvendor problem
    for demand in demand_scenarios:
        
        # Calculate the solution for this scenario
        solution = multi_product_newsvendor(6, demand, prices, costs, salvages, alpha_data)

        # Store the solution for this scenario
        saa_solutions.append(solution)

    # Average the solutions to get the final allocation
    final_allocation = np.mean(saa_solutions, axis=0)

    # Store the final order quantities
    final_order_quantities_DT.append(pd.DataFrame(final_allocation))


AttributeError: 'DataFrame' object has no attribute 'flatten'

# Solving Newsvendor (model-based)

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm

scenario_size = 100

# Initialize an empty list to store the final order quantities
final_order_quantities_ANN_model = []

# Calculate the forecast error for ANN on the prediction of the training data
target_train_flatten = target_train.values.flatten()
train_pred_ANN_flatten = train_pred_ANN.flatten()
forecast_error_ANN = target_train_flatten - train_pred_ANN_flatten
forecast_error_ANN_std = np.std(forecast_error_ANN) 

# Loop over each week in data_test
for i in range(len(data_test)):

    # Create Demand Scenarios for this week
    demand_scenarios = target_pred_ANN[i] + np.random.normal(loc=0, scale=forecast_error_ANN_std, size=scenario_size)

    # Initialize a list to store the solutions for each scenario
    saa_solutions = []

    # For each demand scenario, solve the newsvendor problem
    for demand in demand_scenarios:
        
        # Calculate the solution for this scenario
        solution = multi_product_newsvendor(6, demand, prices, costs, salvages, alpha_data)

        # Store the solution for this scenario
        saa_solutions.append(solution)

    # Average the solutions to get the final allocation
    final_allocation = np.mean(saa_solutions, axis=0)

    # Store the final order quantities
    final_order_quantities_ANN_model.append(final_allocation)

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm

scenario_size = 100

# Initialize an empty list to store the final order quantities
final_order_quantities_DT_model = []

# Calculate the forecast error for ANN on the prediction of the training data
target_train_flatten = target_train.values.flatten()
train_pred_DT_flatten = train_pred_DT.flatten()
forecast_error_DT = target_train_flatten - train_pred_DT_flatten
forecast_error_DT_std = np.std(forecast_error_DT)

# Loop over each week in data_test
for i in range(len(data_test)):

    # Create Demand Scenarios for this week
    demand_scenarios = target_pred_DT[i] + np.random.normal(loc=0, scale=forecast_error_DT_std, size=scenario_size)

    # Initialize a list to store the solutions for each scenario
    saa_solutions = []

    # For each demand scenario, solve the newsvendor problem
    for demand in demand_scenarios:
        
        # Calculate the solution for this scenario
        solution = multi_product_newsvendor(6, demand, prices, costs, salvages, alpha_data)

        # Store the solution for this scenario
        saa_solutions.append(solution)

    # Average the solutions to get the final allocation
    final_allocation = np.mean(saa_solutions, axis=0)

    # Store the final order quantities
    final_order_quantities_DT_model.append(final_allocation)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 35640.25621339922
            Iterations: 31
            Function evaluations: 152
            Gradient evaluations: 31
Optimization terminated successfully    (Exit mode 0)
            Current function value: 35077.34587722396
            Iterations: 27
            Function evaluations: 113
            Gradient evaluations: 27
Optimization terminated successfully    (Exit mode 0)
            Current function value: 34777.528227857736
            Iterations: 27
            Function evaluations: 117
            Gradient evaluations: 27
Optimization terminated successfully    (Exit mode 0)
            Current function value: 34458.34775033546
            Iterations: 32
            Function evaluations: 156
            Gradient evaluations: 32
Optimization terminated successfully    (Exit mode 0)
            Current function value: 36170.57398959826
            Iterations: 26
            Function eva

# Evaluation

In [ ]:
# Loop over each week in target_test
overall_costs_ANN = 0
overall_costs_DT = 0
overall_costs_ANN_model = 0
overall_costs_DT_model = 0

for i in range(len(target_test)):

    # Calculate understock and overstock costs
    cost_ANN = 0
    cost_DT = 0
    cost_ANN_model = 0
    cost_DT_model = 0

    # Costs for non-parametric
    if final_order_quantities_ANN[i] < target_test.values[i]:
        cost_ANN = (p - c) * (target_test.values[i] - np.round(final_order_quantities_ANN[i]))
    if final_order_quantities_ANN[i] > target_test.values[i]:
        cost_ANN = (c - s) * (np.round(final_order_quantities_ANN[i]) - target_test.values[i])
    if final_order_quantities_DT[i] < target_test.values[i]:
        cost_DT = (p - c) * (target_test.values[i] - np.round(final_order_quantities_DT[i]))
    if final_order_quantities_DT[i] > target_test.values[i]:
        cost_DT = (c - s) * (np.round(final_order_quantities_DT[i]) - target_test.values[i])
    # Costs for model-based
    if final_order_quantities_ANN_model[i] < target_test.values[i]:
        cost_ANN_model = (p - c) * (target_test.values[i] - np.round(final_order_quantities_ANN_model[i]))
    if final_order_quantities_ANN_model[i] > target_test.values[i]:
        cost_ANN_model = (c - s) * (np.round(final_order_quantities_ANN_model[i]) - target_test.values[i])
    if final_order_quantities_DT_model[i] < target_test.values[i]:
        cost_DT_model = (p - c) * (target_test.values[i] - np.round(final_order_quantities_DT_model[i]))
    if final_order_quantities_DT_model[i] > target_test.values[i]:
        cost_DT_model = (c - s) * (np.round(final_order_quantities_DT_model[i]) - target_test.values[i])

    # Calculate the total costs for the week
    overall_costs_ANN += cost_ANN
    overall_costs_DT += cost_DT
    overall_costs_ANN_model += cost_ANN_model
    overall_costs_DT_model += cost_DT_model
   

# Print the overall profit
print('Overall costs for ANN (non-parametric): ', int(overall_costs_ANN))
print('Overall costs for DT (non-parametric): ', int(overall_costs_DT))
print('Overall costs for ANN (model-based): ', int(overall_costs_ANN_model))
print('Overall costs for DT (model-based): ', int(overall_costs_DT_model))